# Import Packages and Load Dataset

In [21]:
import nengo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib widget

from matplotlib import cm
from scipy.stats import gennorm
from sklearn.decomposition import PCA
from IPython import display
from TacSet import TacSet


# Load dataset
streams = np.load('touch_stream.npz')
[sSharp, sRound, sPlateau] = [streams[file] for file in streams.files]
labels = ['sharp', 'round', 'plateau']

def compute_vec(data):
    pca = PCA(n_components=2)
    mat = -np.squeeze(data)
    mat = (mat - np.min(mat))/np.std(mat)
    x, y = np.where(mat > np.mean(mat))
    X = np.squeeze(np.dstack((x, y)))
    x0, y0 = np.ceil(np.mean(X, axis=0))
    pca.fit(X)
    V = pca.components_
    a = np.sqrt(pca.singular_values_)
    V[0, :] *= a[1]
    V[1, :] *= 1.5*a[0]
    origin = np.array([[y0, y0],[x0, x0]])
    return origin, V

# Visualize Dataset

In [22]:
frames = np.min([len(sSharp), len(sRound), len(sPlateau)])

# Visualize data steam
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=100)
im = ax.imshow(sPlateau[0])
origin, V = compute_vec(sPlateau[0])
quiver = ax.quiver(*origin, V[:,0], V[:,1], color=['b','r'], scale=20)
def update(frame):
    im.set_data(sPlateau[frame])
    origin, V = compute_vec(sPlateau[frame])
    quiver.set_offsets(origin.T)
    quiver.set_UVC(V[:,0], V[:,1])
    return [im, quiver]
anim = animation.FuncAnimation(fig=fig, func=update, frames=frames, interval=30, blit=True, repeat=False)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()  

# Generalized Gaussian Spiking Encoding for Tactile Sensors

In [11]:
tacset = TacSet([sSharp, sRound, sPlateau], labels)
sample = tacset[4180]

(15, 15, 1)